# 📊 Notebook 2: Database Setup & 10k Images Processing

## Purpose
Process your 10,000 grocery product images and build a comprehensive database:
- Create SQLite database schema
- Batch process all images with YOLO + OCR
- Extract nutrition facts and allergens
- Build similarity search index
- Generate statistics and insights

## ✨ 100% FREE & OFFLINE
**No API costs!** Uses only:
- Your trained YOLO model
- EasyOCR & PaddleOCR (free)
- Local processing only

## What This Notebook Does
1. ✅ Create database schema
2. ✅ Enhanced OCR for nutrition facts
3. ✅ Allergen detection system (14 EU allergens)
4. ✅ Batch process 10,000 images
5. ✅ Build FAISS similarity index
6. ✅ Generate processing report

## Requirements
- Your trained YOLO model
- 10,000 product images folder

---
## 📦 Step 1: Install Additional Dependencies

In [3]:
# Install packages for database and similarity search
!pip install -q paddleocr  # Better OCR for nutrition tables
!pip install -q paddlepaddle  # PaddleOCR backend
!pip install -q faiss-cpu  # Similarity search
!pip install -q sentence-transformers  # For embeddings
!pip install -q tqdm  # Progress bars
!pip install -q pandas
!pip install -q matplotlib
!pip install -q seaborn

print("✅ All packages installed!")

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


✅ All packages installed!


In [1]:
!pip install -q seaborn

In [2]:
!pip install paddleocr

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 20.3 MB/s  0:00:00
   ---------------------------------------- 0.0/45.5 MB ? eta -:--:--
   --- ------------------------------------ 3.9/45.5 MB 18.0 MB/s eta 0:00:03
   ------ --------------------------------- 7.1/45.5 MB 16.7 MB/s eta 0:00:03
   ---------- ----------------------------- 12.1/45.5 MB 18.9 MB/s eta 0:00:02
   ---------------- ----------------------- 18.9/45.5 MB 22.5 MB/s eta 0:00:02
   ---------------------- ----------------- 26.0/45.5 MB 24.5 MB/s eta 0:00:01
   ----------------------------- ---------- 33.0/45.5 MB 26.2 MB/s eta 0:00:01
   ---------------------------------- ----- 39.3/45.5 MB 26.9 MB/s eta 0:00:01
   ---------------------------------------  45.4/45.5 MB 27.5 MB/s eta 0:00:01
   ---------------------------------------- 45.5/45.5 MB 26.1 MB/s  0:00:01
   ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
   ---------

  You can safely remove it manually.


In [2]:
!pip install sentence-transformers 

In [3]:
!pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


---
## 🔧 Step 2: Import Libraries and Setup

In [1]:
import os
import sqlite3
import json
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
import torch
from tqdm import tqdm
import re
import pickle
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# OCR
import easyocr
from paddleocr import PaddleOCR

# Embeddings and similarity search
from sentence_transformers import SentenceTransformer
import faiss

print("✅ Libraries imported successfully!")

c:\anaconda\envs\ironhack\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries imported successfully!


---
## ⚙️ Step 3: Configuration

**UPDATE THESE PATHS TO MATCH YOUR SETUP**

In [2]:
# ============================================================================
# CONFIGURATION - UPDATE THESE PATHS
# ============================================================================

# Your dataset paths
IMAGES_FOLDER = r"C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\images"
YOLO_WEIGHTS = r"C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\runs\nutrigreen_exp\weights\best.pt"
YOLO_REPO = r"C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5"

# Output paths (inside your db folder)
DATABASE_PATH = r"C:\Users\lokes\Desktop\ironhack\final_project\final_project version2\db\nutrigreen_products.db"
EMBEDDINGS_INDEX_PATH = r"C:\Users\lokes\Desktop\ironhack\final_project\final_project version2\db\product_embeddings.faiss"
EMBEDDINGS_METADATA_PATH = r"C:\Users\lokes\Desktop\ironhack\final_project\final_project version2\db\product_metadata.pkl"
PROCESSING_LOG_PATH = r"C:\Users\lokes\Desktop\ironhack\final_project\final_project version2\db\processing_log.json"

# Processing settings
BATCH_SIZE = 100  # Process images in batches
SAVE_INTERVAL = 50  # Save to database every N images

print("✅ Configuration set")
print(f"\nImages folder: {IMAGES_FOLDER}")
print(f"Database will be saved to: {DATABASE_PATH}")
print(f"\n💡 Using YOLO + OCR only (no API costs!)")
print(f"   Expected speed: ~1-2 seconds per image")
print(f"   Estimated time for 10k images: ~3-5 hours")


✅ Configuration set

Images folder: C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\images
Database will be saved to: C:\Users\lokes\Desktop\ironhack\final_project\final_project version2\db\nutrigreen_products.db

💡 Using YOLO + OCR only (no API costs!)
   Expected speed: ~1-2 seconds per image
   Estimated time for 10k images: ~3-5 hours


---
## 🗄️ Step 4: Create Database Schema

In [3]:
def create_database():
    """
    Create SQLite database with comprehensive schema
    """
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()
    
    # Main products table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        image_path TEXT UNIQUE NOT NULL,
        image_filename TEXT,
        brand TEXT,
        product_name TEXT,
        category TEXT,
        weight TEXT,
        nutriscore TEXT,
        is_organic INTEGER,
        is_vegan_vegetarian INTEGER,
        health_score REAL,
        ocr_text TEXT,
        processing_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    # Detected labels table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS detected_labels (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_id INTEGER,
        label_type TEXT,
        confidence REAL,
        bbox_x1 INTEGER,
        bbox_y1 INTEGER,
        bbox_x2 INTEGER,
        bbox_y2 INTEGER,
        FOREIGN KEY (product_id) REFERENCES products(id)
    )
    ''')
    
    # Nutrition facts table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS nutrition_facts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_id INTEGER,
        calories REAL,
        total_fat REAL,
        saturated_fat REAL,
        carbohydrates REAL,
        sugars REAL,
        protein REAL,
        salt REAL,
        fiber REAL,
        serving_size TEXT,
        FOREIGN KEY (product_id) REFERENCES products(id)
    )
    ''')
    
    # Allergens table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS allergens (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_id INTEGER,
        allergen_name TEXT,
        allergen_type TEXT,
        confidence REAL,
        FOREIGN KEY (product_id) REFERENCES products(id)
    )
    ''')
    
    # Processing errors log
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS processing_errors (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        image_path TEXT,
        error_message TEXT,
        error_type TEXT,
        timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    # Create indexes for faster queries
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_brand ON products(brand)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_nutriscore ON products(nutriscore)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_health_score ON products(health_score)')
    
    conn.commit()
    conn.close()
    
    print("✅ Database created successfully!")
    print(f"   Location: {DATABASE_PATH}")
    print("\n📊 Tables created:")
    print("   - products (main product data)")
    print("   - detected_labels (YOLO detections)")
    print("   - nutrition_facts (nutritional information)")
    print("   - allergens (allergen warnings)")
    print("   - processing_errors (error logs)")

# Create the database
create_database()

✅ Database created successfully!
   Location: C:\Users\lokes\Desktop\ironhack\final_project\final_project version2\db\nutrigreen_products.db

📊 Tables created:
   - products (main product data)
   - detected_labels (YOLO detections)
   - nutrition_facts (nutritional information)
   - allergens (allergen warnings)
   - processing_errors (error logs)


---
## 🔍 Step 5: Enhanced OCR for Nutrition Facts

In [4]:
class NutritionExtractor:
    """
    Extract nutrition facts from product images
    Uses both EasyOCR and PaddleOCR (if available)
    """

    def __init__(self):
        print("🔧 Initializing OCR engines...")
        # EasyOCR
        self.easy_reader = easyocr.Reader(
            ['en', 'fr', 'de', 'es'],
            gpu=torch.cuda.is_available(),
            verbose=False
        )

        # PaddleOCR (optional)
        try:
            # remove show_log – not supported in your PaddleOCR version
            self.paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')
            print("✅ PaddleOCR initialized")
        except Exception as e:
            self.paddle_ocr = None
            print(f"⚠️ PaddleOCR not available, continuing with EasyOCR only. Reason: {e}")

        print("✅ OCR engines ready")

        # Nutrition keywords patterns (multi-language)
        self.nutrition_patterns = {
            'calories': r'(calories?|kcal|energie|energy)[:\s]*([0-9]+[.,]?[0-9]*)',
            'fat': r'(fat|fett|graisses?|matières?\s*grasses?)[:\s]*([0-9]+[.,]?[0-9]*)\s*g',
            'saturated_fat': r'(saturated|gesättigt|saturées?)[:\s]*([0-9]+[.,]?[0-9]*)\s*g',
            'carbs': r'(carbohydrat|glucid|kohlenhydrat)[:\s]*([0-9]+[.,]?[0-9]*)\s*g',
            'sugars': r'(sugar|zucker|sucres?)[:\s]*([0-9]+[.,]?[0-9]*)\s*g',
            'protein': r'(protein|eiweiß|protéin)[:\s]*([0-9]+[.,]?[0-9]*)\s*g',
            'salt': r'(salt|salz|sel)[:\s]*([0-9]+[.,]?[0-9]*)\s*g',
            'fiber': r'(fiber|fibre|ballaststoff)[:\s]*([0-9]+[.,]?[0-9]*)\s*g'
        }

    def extract_nutrition(self, image):
        """
        Extract nutrition facts from image

        Args:
            image: PIL Image

        Returns:
            dict: Nutrition facts
        """
        try:
            img_array = np.array(image)

            text_parts = []

            # Use PaddleOCR if available
            if self.paddle_ocr is not None:
                paddle_result = self.paddle_ocr.ocr(img_array, cls=True)
                if paddle_result and paddle_result[0]:
                    for line in paddle_result[0]:
                        text_parts.append(line[1][0])

            # (Optional) You could also add EasyOCR text here if you want to merge both
            # easy_results = self.easy_reader.readtext(img_array, detail=0)
            # text_parts.extend(easy_results)

            full_text = ' '.join(text_parts).lower()
            nutrition_data = {}

            for nutrient, pattern in self.nutrition_patterns.items():
                match = re.search(pattern, full_text, re.IGNORECASE)
                if match:
                    try:
                        value = float(match.group(2).replace(',', '.'))
                        nutrition_data[nutrient] = value
                    except:
                        pass

            serving_match = re.search(
                r'(serving|portion)[:\s]*([0-9]+[.,]?[0-9]*)\s*(g|ml)',
                full_text,
                re.IGNORECASE
            )
            if serving_match:
                nutrition_data['serving_size'] = f"{serving_match.group(2)} {serving_match.group(3)}"

            return nutrition_data if nutrition_data else None

        except Exception:
            return None


# Initialize nutrition extractor
nutrition_extractor = NutritionExtractor()


🔧 Initializing OCR engines...
⚠️ PaddleOCR not available, continuing with EasyOCR only. Reason: No module named 'paddle'
✅ OCR engines ready


C:\Users\lokes\AppData\Local\Temp\ipykernel_23676\1230399197.py:19: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  self.paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')


---
## 🚨 Step 6: Allergen Detection System

In [6]:
class AllergenDetector:
    """
    Detect allergens in product text
    Supports 14 EU major allergens in multiple languages
    """
    
    def __init__(self):
        # 14 EU Major Allergens with multi-language patterns
        self.allergens = {
            'gluten': ['gluten', 'wheat', 'weizen', 'blé', 'trigo', 'barley', 'rye', 'oats'],
            'crustaceans': ['crustacean', 'crab', 'lobster', 'shrimp', 'prawn', 'crevette', 'krebstier'],
            'eggs': ['egg', 'ei', 'oeuf', 'huevo'],
            'fish': ['fish', 'fisch', 'poisson', 'pescado'],
            'peanuts': ['peanut', 'erdnuss', 'arachide', 'cacahuete'],
            'soybeans': ['soy', 'soja', 'soybean'],
            'milk': ['milk', 'milch', 'lait', 'leche', 'dairy', 'lactose', 'whey', 'casein'],
            'nuts': ['almond', 'hazelnut', 'walnut', 'cashew', 'pecan', 'brazil nut', 'pistachio', 'macadamia', 'nuss', 'noix'],
            'celery': ['celery', 'sellerie', 'céleri', 'apio'],
            'mustard': ['mustard', 'senf', 'moutarde', 'mostaza'],
            'sesame': ['sesame', 'sesam', 'sésame', 'sésamo'],
            'sulphites': ['sulphite', 'sulfite', 'sulfit', 'sulfito', 'so2'],
            'lupin': ['lupin', 'lupine'],
            'molluscs': ['mollusc', 'mollusk', 'mussel', 'oyster', 'clam', 'weichtier', 'mollusque']
        }
        
        self.warning_patterns = {
            'contains': r'(contains?|enthält|contient|contiene)[:\s]*([^.]+)',
            'may_contain': r'(may contain|kann enthalten|peut contenir|puede contener)[:\s]*([^.]+)'
        }
        
        print(f"✅ Allergen detector initialized ({len(self.allergens)} categories)")
    
    def detect_allergens(self, text):
        """
        Detect allergens in text
        
        Args:
            text: OCR extracted text
        
        Returns:
            list: Detected allergens with type and confidence
        """
        if not text:
            return []
        
        text_lower = text.lower()
        detected = []
        
        # Check for explicit warning phrases
        for warning_type, pattern in self.warning_patterns.items():
            match = re.search(pattern, text_lower, re.IGNORECASE)
            if match:
                warning_text = match.group(2)
                
                for allergen_name, keywords in self.allergens.items():
                    for keyword in keywords:
                        if keyword in warning_text:
                            detected.append({
                                'allergen': allergen_name,
                                'type': warning_type,
                                'confidence': 0.9
                            })
                            break
        
        # Check for allergens in general text (lower confidence)
        for allergen_name, keywords in self.allergens.items():
            if any(d['allergen'] == allergen_name for d in detected):
                continue
            
            for keyword in keywords:
                if keyword in text_lower:
                    detected.append({
                        'allergen': allergen_name,
                        'type': 'possible',
                        'confidence': 0.6
                    })
                    break
        
        return detected

# Initialize allergen detector
allergen_detector = AllergenDetector()

✅ Allergen detector initialized (14 categories)


---
## 🤖 Step 7: Load YOLO Model

In [7]:
import sys
sys.path.insert(0, str(YOLO_REPO))

try:
    yolo_model = torch.hub.load(
        str(YOLO_REPO),
        'custom',
        path=str(YOLO_WEIGHTS),
        source='local',
        force_reload=False
    )
    yolo_model.conf = 0.25
    
    print("✅ YOLO model loaded successfully!")
    print(f"   Weights: {YOLO_WEIGHTS}")
    
except Exception as e:
    print(f"❌ Error loading YOLO model: {e}")
    yolo_model = None

# Label names
LABEL_NAMES = {
    0: 'NutriScore_A',
    1: 'NutriScore_B',
    2: 'NutriScore_C',
    3: 'NutriScore_D',
    4: 'NutriScore_E',
    5: 'BIO',
    6: 'V-Label'
}

YOLOv5  v7.0-450-g781b9d57 Python-3.11.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 157 layers, 7029004 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


✅ YOLO model loaded successfully!
   Weights: C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\runs\nutrigreen_exp\weights\best.pt


---
## 🎯 Step 8: Main Processing Function

In [8]:
def process_single_image(image_path, yolo_model):
    """
    Process a single product image
    YOLO + OCR + Nutrition + Allergens (no API needed!)
    
    Args:
        image_path: Path to image file
        yolo_model: Loaded YOLO model
    
    Returns:
        dict: Processed product data
    """
    try:
        # Load image
        image = Image.open(image_path).convert('RGB')
        
        # 1. YOLO Detection
        yolo_results = yolo_model(np.array(image))
        detections = []
        
        for pred in yolo_results.pred[0]:
            x1, y1, x2, y2, conf, cls = pred.tolist()
            detections.append({
                'bbox': [int(x1), int(y1), int(x2), int(y2)],
                'confidence': float(conf),
                'class': int(cls),
                'label': LABEL_NAMES[int(cls)]
            })
        
        # 2. OCR Text Extraction
        ocr_text = ''
        ocr_results = []
        try:
            ocr_results = nutrition_extractor.easy_reader.readtext(np.array(image))
            ocr_text = ' '.join([text for _, text, _ in ocr_results])
        except:
            pass
        
        # 3. Extract brand and product name
        brand = None
        product_name = None
        if ocr_results:
            sorted_results = sorted(ocr_results, key=lambda x: x[0][0][1])
            if len(sorted_results) > 0:
                brand = sorted_results[0][1]
            if len(sorted_results) > 1:
                product_name = sorted_results[1][1]
        
        # 4. Nutrition Facts Extraction
        nutrition_data = nutrition_extractor.extract_nutrition(image)
        
        # 5. Allergen Detection
        allergens = allergen_detector.detect_allergens(ocr_text)
        
        # 6. Calculate metadata
        nutriscore = None
        is_organic = 0
        is_vegan_vegetarian = 0
        health_score = 50.0
        
        for det in detections:
            if 'NutriScore' in det['label']:
                nutriscore = det['label']
                score_map = {'NutriScore_A': 100, 'NutriScore_B': 80, 'NutriScore_C': 60, 
                           'NutriScore_D': 40, 'NutriScore_E': 20}
                health_score = score_map.get(nutriscore, 50)
            elif det['label'] == 'BIO':
                is_organic = 1
            elif det['label'] == 'V-Label':
                is_vegan_vegetarian = 1
        
        return {
            'image_path': str(image_path),
            'image_filename': Path(image_path).name,
            'brand': brand,
            'product_name': product_name,
            'nutriscore': nutriscore,
            'is_organic': is_organic,
            'is_vegan_vegetarian': is_vegan_vegetarian,
            'health_score': health_score,
            'ocr_text': ocr_text,
            'detections': detections,
            'nutrition_data': nutrition_data,
            'allergens': allergens,
            'success': True
        }
        
    except Exception as e:
        return {
            'image_path': str(image_path),
            'success': False,
            'error': str(e)
        }

print("✅ Processing function ready")

✅ Processing function ready


---
## 💾 Step 9: Database Insert Functions

In [10]:
def save_product_to_db(product_data):
    """
    Save processed product to database
    """
    if not product_data.get('success'):
        conn = sqlite3.connect(DATABASE_PATH)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO processing_errors (image_path, error_message, error_type)
            VALUES (?, ?, ?)
        ''', (product_data['image_path'], product_data.get('error', 'Unknown'), 'processing'))
        conn.commit()
        conn.close()
        return None
    
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()
    
    try:
        cursor.execute('''
            INSERT INTO products (
                image_path, image_filename, brand, product_name,
                nutriscore, is_organic, is_vegan_vegetarian, health_score, ocr_text
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            product_data['image_path'],
            product_data['image_filename'],
            product_data.get('brand'),
            product_data.get('product_name'),
            product_data.get('nutriscore'),
            product_data['is_organic'],
            product_data['is_vegan_vegetarian'],
            product_data['health_score'],
            product_data.get('ocr_text')
        ))
        
        product_id = cursor.lastrowid
        
        # Insert detected labels
        for detection in product_data.get('detections', []):
            cursor.execute('''
                INSERT INTO detected_labels (
                    product_id, label_type, confidence,
                    bbox_x1, bbox_y1, bbox_x2, bbox_y2
                ) VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (
                product_id,
                detection['label'],
                detection['confidence'],
                *detection['bbox']
            ))
        
        # Insert nutrition facts
        nutrition = product_data.get('nutrition_data')
        if nutrition:
            cursor.execute('''
                INSERT INTO nutrition_facts (
                    product_id, calories, total_fat, saturated_fat,
                    carbohydrates, sugars, protein, salt, fiber, serving_size
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                product_id,
                nutrition.get('calories'),
                nutrition.get('fat'),
                nutrition.get('saturated_fat'),
                nutrition.get('carbs'),
                nutrition.get('sugars'),
                nutrition.get('protein'),
                nutrition.get('salt'),
                nutrition.get('fiber'),
                nutrition.get('serving_size')
            ))
        
        # Insert allergens
        for allergen in product_data.get('allergens', []):
            cursor.execute('''
                INSERT INTO allergens (
                    product_id, allergen_name, allergen_type, confidence
                ) VALUES (?, ?, ?, ?)
            ''', (
                product_id,
                allergen['allergen'],
                allergen['type'],
                allergen['confidence']
            ))
        
        conn.commit()
        return product_id
        
    except Exception as e:
        return None
    finally:
        conn.close()

print("✅ Database save functions ready")

✅ Database save functions ready


---
## 📁 Step 10: Get Image Files

In [11]:
from pathlib import Path

image_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
images_path = Path(IMAGES_FOLDER)

if images_path.exists():
    image_files = []
    for ext in image_extensions:
        image_files.extend(images_path.rglob(f'*{ext}'))

    # Deduplicate (important on Windows with case-insensitive FS)
    image_files = list(set(image_files))

    print(f"✅ Found {len(image_files)} images")
    print(f"   Location: {IMAGES_FOLDER}")

    if len(image_files) > 0:
        print(f"\n📂 Sample files:")
        for i, img in enumerate(image_files[:5], 1):
            print(f"   {i}. {img.name}")
else:
    print(f"❌ Images folder not found: {IMAGES_FOLDER}")


✅ Found 7271 images
   Location: C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\images

📂 Sample files:
   1. 3390390003185.jpg
   2. nutriScoreD (772).jpg
   3. 4104420054004.jpg
   4. nutriScoreE (31).jpg
   5. OpenFoodFacts405.jpg


---
## 🚀 Step 11: BATCH PROCESSING



In [12]:
def batch_process_images(image_files, yolo_model):
    """
    Process all images in batches
    """
    stats = {
        'total': len(image_files),
        'processed': 0,
        'successful': 0,
        'failed': 0,
        'with_nutrition': 0,
        'with_allergens': 0,
        'start_time': datetime.now()
    }
    
    print(f"\n{'='*70}")
    print(f"STARTING BATCH PROCESSING - 100% FREE & OFFLINE")
    print(f"{'='*70}")
    print(f"Total images: {stats['total']}")
    print(f"Started at: {stats['start_time'].strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'='*70}\n")
    
    for img_path in tqdm(image_files, desc="Processing images"):
        try:
            product_data = process_single_image(img_path, yolo_model)
            save_product_to_db(product_data)
            
            stats['processed'] += 1
            
            if product_data.get('success'):
                stats['successful'] += 1
                if product_data.get('nutrition_data'):
                    stats['with_nutrition'] += 1
                if product_data.get('allergens'):
                    stats['with_allergens'] += 1
            else:
                stats['failed'] += 1
            
            if stats['processed'] % 100 == 0:
                elapsed = (datetime.now() - stats['start_time']).total_seconds()
                rate = stats['processed'] / elapsed
                remaining = (stats['total'] - stats['processed']) / rate if rate > 0 else 0
                
                print(f"\n📊 Progress: {stats['processed']}/{stats['total']}")
                print(f"   Success: {stats['successful']} | Failed: {stats['failed']}")
                print(f"   Rate: {rate:.2f} images/sec | ETA: {remaining/60:.1f} min\n")
        
        except Exception as e:
            stats['failed'] += 1
            stats['processed'] += 1
    
    stats['end_time'] = datetime.now()
    stats['duration'] = (stats['end_time'] - stats['start_time']).total_seconds()
    return stats

print("✅ Batch processing function ready")
print("\n⚠️ Run the next cell to start processing!")

✅ Batch processing function ready

⚠️ Run the next cell to start processing!


### ▶️ START PROCESSING

In [13]:
if yolo_model and len(image_files) > 0:
    processing_stats = batch_process_images(image_files, yolo_model)
    
    with open(PROCESSING_LOG_PATH, 'w') as f:
        json.dump({
            **processing_stats,
            'start_time': processing_stats['start_time'].isoformat(),
            'end_time': processing_stats['end_time'].isoformat()
        }, f, indent=2)
    
    print("\n" + "="*70)
    print("PROCESSING COMPLETE!")
    print("="*70)
    print(f"Total processed: {processing_stats['processed']}")
    print(f"Successful: {processing_stats['successful']}")
    print(f"Failed: {processing_stats['failed']}")
    print(f"With nutrition data: {processing_stats['with_nutrition']}")
    print(f"With allergens: {processing_stats['with_allergens']}")
    print(f"Duration: {processing_stats['duration']/60:.1f} minutes")
    print(f"Average: {processing_stats['processed']/processing_stats['duration']:.2f} images/sec")
    print("="*70)
else:
    print("❌ Cannot start - check YOLO model and image files")


STARTING BATCH PROCESSING - 100% FREE & OFFLINE
Total images: 7271
Started at: 2025-11-29 19:58:50



Processing images:   0%|          | 0/7271 [00:00<?, ?it/s]C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:   0%|          | 1/7271 [00:03<7:34:46,  3.75s/it]C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:   0%|          | 2/7271 [00:04<3:35:20,  1.78s/it]C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:   0%|          | 3/7271 [00:06<3:40:08,  1.82s/it


📊 Progress: 100/7271
   Success: 100 | Failed: 0
   Rate: 0.41 images/sec | ETA: 293.5 min



C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:   1%|▏         | 101/7271 [04:07<6:43:31,  3.38s/it]C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:   1%|▏         | 102/7271 [04:08<5:02:33,  2.53s/it]C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:   1%|▏         | 103/7271 [04:10<4:38:43,  2.33s/it]C:\Users\lokes\Desktop\ironhack\final_project\datase


📊 Progress: 200/7271
   Success: 200 | Failed: 0
   Rate: 0.53 images/sec | ETA: 221.3 min



...


📊 Progress: 7100/7271
   Success: 7100 | Failed: 0
   Rate: 0.15 images/sec | ETA: 19.3 min



C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:  98%|█████████▊| 7101/7271 [13:20:44<02:44,  1.03it/s]C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:  98%|█████████▊| 7102/7271 [13:20:45<03:02,  1.08s/it]C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:  98%|█████████▊| 7103/7271 [13:20:46<02:49,  1.01s/it]C:\Users\lokes\Desktop\ironhack\final_project\


📊 Progress: 7200/7271
   Success: 7200 | Failed: 0
   Rate: 0.15 images/sec | ETA: 8.0 min



C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:  99%|█████████▉| 7201/7271 [13:30:00<04:26,  3.81s/it]C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:  99%|█████████▉| 7202/7271 [13:30:00<03:15,  2.84s/it]C:\Users\lokes\Desktop\ironhack\final_project\dataset\dataset\yolov5\models\common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing images:  99%|█████████▉| 7203/7271 [13:30:01<02:27,  2.16s/it]C:\Users\lokes\Desktop\ironhack\final_project\


PROCESSING COMPLETE!
Total processed: 7271
Successful: 7271
Failed: 0
With nutrition data: 0
With allergens: 3076
Duration: 816.0 minutes
Average: 0.15 images/sec


---
## 📊 Step 12: Database Statistics

In [14]:
conn = sqlite3.connect(DATABASE_PATH)

stats_df = pd.read_sql_query('''
    SELECT 
        COUNT(*) as total_products,
        COUNT(DISTINCT brand) as unique_brands,
        SUM(is_organic) as organic_count,
        SUM(is_vegan_vegetarian) as vegan_count,
        AVG(health_score) as avg_health_score
    FROM products
''', conn)

print("\n" + "="*70)
print("DATABASE STATISTICS")
print("="*70)
print(stats_df.to_string(index=False))

nutriscore_df = pd.read_sql_query('''
    SELECT nutriscore, COUNT(*) as count
    FROM products WHERE nutriscore IS NOT NULL
    GROUP BY nutriscore ORDER BY nutriscore
''', conn)

print("\n📊 NutriScore Distribution:")
print(nutriscore_df.to_string(index=False))

allergen_df = pd.read_sql_query('''
    SELECT allergen_name, COUNT(*) as count
    FROM allergens GROUP BY allergen_name
    ORDER BY count DESC LIMIT 10
''', conn)

print("\n🚨 Top 10 Allergens:")
print(allergen_df.to_string(index=False))

conn.close()


DATABASE STATISTICS
 total_products  unique_brands  organic_count  vegan_count  avg_health_score
           7271           4465           2330          890         59.062027

📊 NutriScore Distribution:
  nutriscore  count
NutriScore_A   1256
NutriScore_B   1114
NutriScore_C    866
NutriScore_D   1013
NutriScore_E    962

🚨 Top 10 Allergens:
allergen_name  count
         eggs   2060
         milk    558
       gluten    539
     soybeans    339
         nuts    253
       sesame     92
      mustard     61
         fish     50
      peanuts     45
     molluscs     35


---
## 🔍 Step 13: Build Similarity Search Index

In [15]:
print("🔨 Building similarity search index...\n")

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded")

conn = sqlite3.connect(DATABASE_PATH)
products_df = pd.read_sql_query('''
    SELECT id, brand, product_name, nutriscore, ocr_text
    FROM products
''', conn)
conn.close()

print(f"📊 Creating embeddings for {len(products_df)} products...")

texts = []
for _, row in products_df.iterrows():
    text_parts = []
    if row['brand']: text_parts.append(f"Brand: {row['brand']}")
    if row['product_name']: text_parts.append(f"Product: {row['product_name']}")
    if row['nutriscore']: text_parts.append(f"NutriScore: {row['nutriscore']}")
    if row['ocr_text']: text_parts.append(row['ocr_text'][:200])
    texts.append(' '.join(text_parts) if text_parts else 'Unknown product')

embeddings = embedding_model.encode(texts, show_progress_bar=True, batch_size=32)
print(f"✅ Generated {len(embeddings)} embeddings")

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))

faiss.write_index(index, EMBEDDINGS_INDEX_PATH)
print(f"✅ FAISS index saved: {EMBEDDINGS_INDEX_PATH}")

metadata = products_df.to_dict('records')
with open(EMBEDDINGS_METADATA_PATH, 'wb') as f:
    pickle.dump(metadata, f)
print(f"✅ Metadata saved: {EMBEDDINGS_METADATA_PATH}")

print("\n🎉 Similarity search ready!")

🔨 Building similarity search index...

✅ Embedding model loaded
📊 Creating embeddings for 7271 products...


Batches: 100%|██████████| 228/228 [00:08<00:00, 28.39it/s]


✅ Generated 7271 embeddings
✅ FAISS index saved: C:\Users\lokes\Desktop\ironhack\final_project\final_project version2\db\product_embeddings.faiss
✅ Metadata saved: C:\Users\lokes\Desktop\ironhack\final_project\final_project version2\db\product_metadata.pkl

🎉 Similarity search ready!


---
## ✅ Summary

### 🎉 What We Accomplished:
1. ✅ Created comprehensive SQLite database
2. ✅ Processed 10,000 product images (100% FREE!)
3. ✅ Extracted nutrition facts with OCR
4. ✅ Detected allergens (14 categories)
5. ✅ Built FAISS similarity search index
6. ✅ Generated statistics

### 💰 Total Cost: $0.00
No API tokens used! Everything runs locally.

### 📁 Files Created:
- `nutrigreen_products.db` - Database with all products
- `product_embeddings.faiss` - Search index
- `product_metadata.pkl` - Product metadata
- `processing_log.json` - Statistics

### 🚀 Ready for Notebook 3!
Next: Advanced features (comparison, calculator, recommendations)